In [54]:
import numpy as np
from ann import ANN

fileName = "testing.csv"

weights = [9.997745999974152, -6.037329401407339, 5.575748741393019, 5.196236054149269, -6.5452486608431375, -7.710246135719355, -0.4000532513348616, 4.546188432326241, -6.986034385936812, 3.6254777097587407, -5.370499098353402, -1.8000871849140574, -4.824101704748764, 9.411211307778443, 0.4455964173710253, 10.313731745153397, 3.676341077658349, 8.995525787207614, 3.5266127889454735, -9.016215322623486, -4.549089363119515, 10.793809253560749, 0.9242491684725371, -6.9015699716613845, -1.7418416614788832, -8.114283058730424, -1.287772590558646, -7.066938573097747, -1.234508169926983, 5.041903789424927, -4.290602199531993, 5.914897442702895, 7.111666320144199, 9.610893830085237, 3.2341069530055746, 6.799152870815522, 3.044461845641069, -8.008601913890981, -3.9096375778650057, 8.735430142008624, 7.568294029901349, -3.497602452226844, 4.665181063757195, -0.5562497231549858, -3.882117692434748, 1.7167030741379516, 8.817338115684286, 8.412168036664214, -6.729542187067986, -8.584741533876349, -10.523307853574098, -5.106057938061306, 6.97583391489255, 3.037087859754779, -2.2184382117708092, -5.022949876123636, 8.714748349031387, -8.780984029375828, 6.713462120747756, 2.5520875900650077, -2.0262160936059086, 4.578203812558475, -9.460118446119425, -0.242017887267224, -8.491384874912518, 8.365368896461828, -8.475779807806767, 4.972041808789118, -7.839089269274506, 1.5509153220961016, -0.759871134882695, -11.674238347634857, -2.6952356310879413, 5.267059863839125, -6.910592619967044, 5.306596086602212, 0.21701524079281476, -6.135899362073868, -0.948316030478377, -2.6327434399316503, -8.4345584943313, 13.128767540428566, 4.366097528686048, 0.07279165231290285, 12.440741535859528, -4.158117166432806, -4.822609573196184, -2.4119943125888943, -0.042204858519387045, -4.079029970134839, -3.614371846063378, -11.509417474199592, -3.3894805747266026, 1.7309657455043126, -8.391450746534964, 4.230480361845339, -4.371533945706552, -6.54462915868204, -10.399823822776746, -6.443592725707155, -1.561943276318054, 7.818169552013434, -6.862075529942159, -5.679518259086267, -1.0287630054368528, 0.19951147829010002, -4.183427993426755, -4.146404272865571, -5.143355693246033, -6.319767366972423, 0.4392829614231317, 2.978059631608498, -6.370311056946601, -6.753138853505943, 2.720425975814141, 6.833675453725682, 0.5613274297945141, 0.8475538929485548, 8.889338335509144, -4.031942941111332, 2.482153093367806, 5.946589283706151, 2.0905500241637007, 5.038184914436523, 0.07715817350005594, -0.9157079291952925, -8.54452118602213, 3.7933705022163715, 3.6988473769984163, 9.922415457280168, 5.513882412780289, 6.126934306796564, 0.9898754335237632, 4.187278773992704, 0.1306574127117886, -7.303199220489019, -1.6513379605404137, 7.72468965536321, 4.333809439958649, 0.8923596777140309, 6.723924793645043, 0.4882161676459825, 6.728195379973083, 1.0062872446282198, -0.9611600965390013, -7.096202497840369, -6.5270945977287695, 8.803216212557638, 10.95919898935745, -7.422711124413633, 3.819850141165179, -1.3464230935744186, 2.3071050756334195, 0.9485488855982872, -4.341880540471459, 11.560188943011054, -6.659673211998888, 5.882788372245, -6.145944872516668, 2.456221619533142, 0.31359136475843985, 8.266903483137732, 1.3344130442231577, 7.111094962226667, -4.339529929214126, -1.9279567248385305, -1.0595199976955374, -0.8692070704411629, -7.572679910468938, -0.9056868404790601, 9.977776508023602, -5.238040098738821, 4.7133958608801985, -9.01896660779756, 4.2479472049424345, -9.814162089124999, 4.658585700583217, -8.212220772919219, 2.6672810650697887, -6.994803364822677, 4.586003866433696, -0.33165246251889746, -7.792445010948774, 4.1396374814262105, -2.499826949344765, -7.5807040237246595, 6.257951667939299, -3.5440748602129046, -3.7520751560837255, -4.2247155583318525, 10.725572216873282, 5.3334794236376695, 5.082917961037561, -3.2920518334686397, -1.1967456759990307, -7.1713080313086, -9.365871723771397, -7.3884978481283925, -4.807081519653328, -6.816904941205385, 10.797768446542864, -0.26466629822263676, -5.343376906115688, -8.631378471454818, -0.8124227082431894, -5.166625227955257, -1.197550597700595, 7.800058686254363, -4.199536034784847, -5.423009114435031, -1.5372866981081266, 0.26884044758318315, -3.7375546509666795, 4.354790038077422, -1.2419401585795025, 3.1379682459870577, 0.23788912611481683, -6.035053573251014, -0.05851972745195433, 6.363532731481454, -1.8329700930159163, 1.6004528051539078, -2.634501441897203, -4.3379566640512905, -0.29054474458956214, 3.785222540398016, -5.137978060929698, 2.1382042284273783, -3.6855808676079516, -1.852910510573265, 8.03718265871433, 4.741997528660319, -9.56180784800323, 7.185350145507208, 0.48329445460881126, 1.427706544829972, 2.8875898538925058, -2.9265431796517243, 2.2311742852617407, 6.1358906211076745, 6.484376261915347, -2.1016183244993165, 5.109479023570073, 3.524117270299196, 3.976000822958861]
#weights=[-87.46554726557201, -16.356639560711052, 86.29709617763675, 71.16875886463859, -44.05130724111197, 46.27304010434485, 61.27005442566909, 8.163160494597655, 46.59122784292944, 2.455822652461255, -106.5029289014251, -14.401230797847266, 62.257594997357806, 44.637763053177906, -22.300364670766378, -13.04067285972872, -57.390738780447535, -48.47680769485232, 97.16607589371515, 3.189555893865565, 107.93767449703456, 9.714060083383517, 41.385599043808796, 32.51785727809452, -9.463832626588042, -1.3141253203749645, -64.93912393955407, -91.40284725263278, 94.86045320712967, -15.53772463112995, 43.23148580319236, -54.141645679024094, -34.247384440569306, 16.009404681617347, 32.290650704655896, -14.576787064751198, -33.64940233339766, -113.7094461364188, 61.01150237682433, 0.08851109599003859, -103.37744443614314, 25.976530611812187, -82.98487960097654, 7.458609073351389, -14.987001096522816, -13.534697374850593, 1.044006840599236]


lat=np.genfromtxt(fileName, dtype=float, delimiter=",",skip_header=1, usecols=0)
lon=np.genfromtxt(fileName, dtype=float, delimiter=",",skip_header=1, usecols=1)
time=np.genfromtxt(fileName, dtype=float, delimiter=",",skip_header=1, usecols=2)
call_dur=np.genfromtxt(fileName, dtype=float, delimiter=",",skip_header=1, usecols=3)
call_type=np.genfromtxt(fileName, dtype=int, delimiter=",",skip_header=1, usecols=4)
response_time=np.genfromtxt(fileName, dtype=float, delimiter=",",skip_header=1, usecols=5)


inputs = np.column_stack((lat,lon,time,call_dur))
outputs = np.column_stack((call_type,response_time))



ann = ANN(weights)
ann_out_ctype = []
ann_out_rtime = []
for i in range(0,inputs.shape[0]):
    out=ann.evaluate(inputs[i])
    ann_out_ctype.append(out[0])
    ann_out_rtime.append(out[1])
ann_out_ctype = np.around(ann_out_ctype)
print(ann_out_rtime)

correct = 0
for i in range(len(ann_out_ctype)):
    if (ann_out_ctype[i]==outputs[i][0]):
        correct = correct+1

ctype_correct = (float(correct)/len(ann_out_ctype))*100
print("correct: ",correct)
print("percent: ", ctype_correct)

response_err = []
for i in range(len(ann_out_rtime)):
    response_err.append(ann_out_rtime[i] - outputs[i][1])

print("mean response prediction error: ",np.mean(response_err))
print("response prediction error st dev: ", np.std(response_err))

ValueError: cannot reshape array of size 206 into shape (2,4)